In [5]:
import sys
sys.path.insert(0,'../../src')

from src.config import app_config, ProductionConfig

prod_config = ProductionConfig()
print('api_key', app_config.BINANCE_API_KEY)
print('api_secret', app_config.BINANCE_API_SECRET)

api_key KA3SYnSVeaDull64B04HZiVtwGH313pTMMwwK7CLhozQNGWVdtntzRuZJ72199BR
api_secret ENY17NqeBrnR0ZngVDurtvbuwFCS2KkAnMhYijVNpd3urkCamOINNWa2cpiaNrH8


In [3]:
from src.service.exchange import ExchangeService

exchange = ExchangeService().get_exchange('Binance', app_config.BINANCE_API_KEY, app_config.BINANCE_API_SECRET)
prod = ExchangeService().get_exchange('Binance', prod_config.BINANCE_API_KEY, prod_config.BINANCE_API_SECRET)


In [4]:
print(exchange.get_symbol_ticker('ETHUSDT'))
print(prod.get_symbol_ticker('ETHUSDT'))

{'symbol': 'ETHUSDT', 'price': '1804.08000000'}
{'symbol': 'ETHUSDT', 'price': '1804.08000000'}


In [5]:
import pandas as pd
data = exchange.get_symbol_data('BTCUSDT', '1m', 100)

# convert to pandas dataframe
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df

,time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1683974400000,26806.18000000,26812.26000000,26806.17000000,26812.25000000,10.44268000,1683974459999,279964.33535270,441,7.61442000,204143.40642170,0
1,1683974460000,26812.26000000,26812.26000000,26810.59000000,26812.24000000,11.57206000,1683974519999,310261.20587570,296,8.26833000,221680.94184300,0
2,1683974520000,26812.25000000,26812.25000000,26809.75000000,26809.75000000,18.50768000,1683974579999,496199.94725330,271,13.16105000,352846.77654910,0
3,1683974580000,26809.76000000,26810.75000000,26809.75000000,26810.75000000,2.67257000,1683974639999,71652.59203110,199,1.50812000,40433.05629030,0
4,1683974640000,26810.75000000,26810.97000000,26810.74000000,26810.97000000,4.81113000,1683974699999,128990.10738180,232,1.70321000,45664.36418110,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1683980100000,26789.84000000,26804.99000000,26789.83000000,26804.30000000,17.95510000,1683980159999,481189.98543280,658,10.41823000,279182.52094840,0
96,1683980160000,26804.31000000,26817.78000000,26804.30000000,26817.78000000,14.27168000,1683980219999,382624.88711400,679,12.04336000,322880.69364030,0
97,1683980220000,26817.78000000,26826.18000000,26817.78000000,26822.70000000,8.47914000,1683980279999,227439.32214270,587,3.55823000,95438.04928080,0
98,1683980280000,26822.71000000,26822.71000000,26821.54000000,26821.55000000,4.89652000,1683980339999,131334.12426230,301,1.73728000,46596.70201160,0


### Generating load test data

**Users**

In [21]:
import bcrypt
import random
import string

def randomString(stringLength=10) -> str:
    letters = string.ascii_letters
    return ''.join([random.choice(letters) for i in range(stringLength)])

user_query = "INSERT INTO users (id, email, _password, address_id, portfolio_id) VALUES ('{}', '{}', '{}', 1, 1);"

number_of_records = 1000

queries = []
plain_passwords = [randomString() for i in range(number_of_records)]
# salt = bcrypt.gensalt()
# hashed = [bcrypt.hashpw(password.encode('utf-8'), salt) for password in plain_passwords]
emails = [password + '@gmail.com' for password in plain_passwords]
queries = [user_query.format(str(i+2), email, password) for i, (email, password) in enumerate(zip(emails, plain_passwords))]

with open('users.load.sql', 'w+') as f:
    for query in queries:
        f.write(query + '\n')


**Api Keys**

In [8]:
number_of_users = 1000

api_key_query = "INSERT INTO api_keys (user_id, exchange, api_key, api_secret) VALUES ('{}', 'binance', '{}', '{}');"

queries = []
for i in range(1, number_of_users + 1):
    query = api_key_query.format(i, api_key, api_secret)
    queries.append(query)

with open('api_keys.load.sql', 'w+') as f:
    for query in queries:
        f.write(query + '\n')

**Currency Pair Configs**

In [9]:
import random
number_of_records = 100
number_of_users = 1000

currency_pairs = exchange.get_all_symbols().keys()
intervals = ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '8h', '12h', '1d', '1w', '1M']
strategies = ['macd', 'rsi',  'stochastic_oscillator', 'williams_r', 'moving_average_crossover', 'bollinger_bands', 'macd_rsi']

currency_pair_config_query = "INSERT INTO currency_pair_configs (user_id, currency_pair, exchange, `interval`, tradeable, strategy, `limit`, autotrade, stop_loss, take_profit, asset_allocation) VALUES ('{}', '{}', 'binance', '{}', 1, '{}', 1000, 1, 0.3, 0.3, 1);"
portfolio_query = "INSERT INTO portfolios (user_id, total_cost, exchange) VALUES ('{}', '{}', 'binance');"
balance_query = "INSERT INTO balances (portfolio_id, asset, free, locked, total) VALUES ('{}', '{}', 1000.0, 0.0, 1000.0);"

existing_users = 1
# offset by the existing users
user_ids = [str(i + existing_users) for i in range(1, number_of_users)]

queries = []
for i, currency_pair in enumerate(currency_pairs):
    if i > number_of_records:
        break
    
    symbol = currency_pair.replace('/', '') # currency_pair
    asset = currency_pair.split('/')[1] # currency
    
    user_id = user_ids[i]
    
    # Random interval, and strategy
    interval = random.choice(intervals)
    strategy = random.choice(strategies)

    # Create a currency pair config for the user
    queryA = currency_pair_config_query.format(user_id, symbol, interval, strategy)

    # Create a portfolio for the user
    queryB = portfolio_query.format(user_id, 1000)

    # Create a balance for the currency used to buy the asset
    queryC = balance_query.format(i + existing_users + 1, asset)

    # Add the queries to the list
    queries.append(queryA)
    queries.append(queryB)
    queries.append(queryC)

with open('user_portfolios.sql', 'w+') as f:
    for query in queries:
        f.write(query + '\n')


**Autotrades**

In [26]:
autotrades_query = "INSERT INTO autotrades (currency_pair_config_id) VALUES ({});"

existing_records = 3
number_of_records = 100 + existing_records
queries = []
# offset by the number of existing records from mock data
for i in range(existing_records, number_of_records):
    query = autotrades_query.format(i)
    queries.append(query)

with open('autotrades.load.sql', 'w+') as f:
    for query in queries:
        f.write(query + '\n')

**Symbols & Currency pairs**

In [17]:
currency_pairs = set(exchange.get_all_symbols().keys())

pairs_not_in_binance = ['YOYOWETH', 'YOYOWBNB', 'YOYOWBTC']

symbol_query = "INSERT INTO symbols (symbol) VALUES ('{}');"
currency_pair_query = "INSERT INTO currency_pairs (symbol, pair, currency_pair) VALUES ('{}', '{}', '{}');"

symbols = {currency_pair.split('/')[0] for currency_pair in currency_pairs}
symbols.remove('BTC')
symbol_queries = [symbol_query.format(symbol) for symbol in symbols]

currency_pair_queries = []
for currency_pair in currency_pairs:
    if currency_pair == 'BTC/USDT':
        continue

    currency_pair_clean = currency_pair.replace('/', '')
    symbol = currency_pair.split('/')[0]
    pair = currency_pair.split('/')[1]
    query = currency_pair_query.format(symbol, pair, currency_pair_clean)
    currency_pair_queries.append(query)

with open('symbols.load.sql', 'w+') as f:
    for query in symbol_queries:
        f.write(query + '\n')
    
    for query in currency_pair_queries:
        f.write(query + '\n')

NameError: name 'symbols_not_in' is not defined

## Bot generation

In [6]:
import sys
sys.path.append('../../')

from src.config import app_config

from src.enums import StrategyType, ExchangeType, Interval
from src.model import UserModel, ApiKeyModel, PortfolioModel, BalanceModel, CurrencyPairConfigModel, SignalModel, StrategyConfigModel

In [7]:
# Queries
user_query = "INSERT INTO users (id, email, _password) VALUES ('{}', '{}', '{}');"

api_key_query = "INSERT INTO api_keys (user_id, exchange, api_key, api_secret) VALUES ('{}', '{}', '{}', '{}');"

portfolio_query = "INSERT INTO portfolios (id, user_id) VALUES ({}, '{}');"

balance_query = "INSERT INTO balances (portfolio_id, asset, free, locked, total) VALUES ('{}', '{}', {}, {}, {});"

currency_pair_config_query = "INSERT INTO currency_pair_configs (user_id, currency_pair, bot_name, exchange, `interval`, strategy, `limit`, is_active, stop_loss, take_profit, allocated_balance, currency_free, currency_locked, asset_free, asset_locked) VALUES ('{}', '{}', '{}', '{}', '{}', '{}', 1000, 1, 0.3, 0.3, {}, {}, 0.0, 0.0, 0.0);"

signal_query = "INSERT INTO signals (currency_pair_config_id, `signal`) VALUES ({}, 'HOLD');"

strategy_config_query = "INSERT INTO strategy_configs (currency_pair_config_id, strategy, `key`, `value`) VALUES ({}, '{}', '{}', {});"

In [9]:
bot_config = {
    'email': 'bot{}@admin.com',
    'password': '$2b$12$F/EmqrDOVllQXDkO9xBXbexbwXVit/GuXOxSvAiQ3NrvPSwJCv7VK',
    'name': 'Bot {} - {}',
    'starting_currency': 100000.0,
    'symbols': ['VIBUSDT', 'GNSUSDT', 'BAKEUSDT', 'EDUUSDT', 'CHRUSDT', 'UNFIUSDT', 'IMXUSDT', 'TRUUSDT', 'DENTUSDT', 'RENUSDT', 'FARMUSDT', 'CFXUSDT', 'STORJUSDT', 'STXUSDT', 'ILVUSDT', 'XVSUSDT', 'TLMUSDT', 'LOOMUSDT', 'ARPAUSDT', 'ALCXUSDT', 'CKBUSDT', 'RNDRUSDT', 'LAZIOUSDT', 'SOLUSDT', 'DOTUSDT', 'AVAUSDT', 'DUSKUSDT', 'BTCUSDT', 'SKLUSDT', 'FLOKIUSDT', 'KNCUSDT', 'DASHUSDT', 'FORUSDT', 'TKOUSDT', 'ETHUSDT', 'ROSEUSDT', 'FXSUSDT', 'FISUSDT', 'GALUSDT', 'MOBUSDT', 'PHBUSDT', 'BNBUSDT', 'BELUSDT', 'KDAUSDT', 'AUDIOUSDT', 'MINAUSDT', 'XRPUSDT', 'CELOUSDT', 'GALAUSDT', 'ATOMUSDT', 'WOOUSDT', 'QNTUSDT', 'SSVUSDT', 'BETAUSDT'],
    'currency': 'USDT',
    'api_key': app_config.BINANCE_API_KEY,
    'api_secret': app_config.BINANCE_API_SECRET,
    'exchange': ExchangeType.BINANCE.name
}

strategies = [
    {
        'strategy': StrategyType.RSI.name,
        'params': {'window': 1},
        'interval': Interval.MINUTE_1.name
    },
]

In [16]:
# Create user
user_queries = []
# Create api key
api_key_queries = []
# Create portfolio
portfolio_queries = []
# Create balances
balance_queries = []
# Create currency pair config
currency_pair_config_queries = []
# Create signal
signal_queries = []
# Create strategy config
strategy_config_queries = []

starting_user_id = 46
config_id = 672 # starting config id
for i, strategy in enumerate(strategies):
    id = starting_user_id + i
    strategy_type = strategy['strategy']
    params = strategy['params']
    interval = strategy['interval']

    # Create user
    query = user_query.format(id, bot_config['email'].format(id), bot_config['password'])
    user_queries.append(query)

    # Create api key
    query = api_key_query.format(id, bot_config['exchange'], bot_config['api_key'], bot_config['api_secret'])
    api_key_queries.append(query)

    # Create portfolio
    query = portfolio_query.format(id, id)
    portfolio_queries.append(query)

    # Create balances
    query = balance_query.format(
        id,
        'USDT',
        0.0,
        bot_config['starting_currency'] * len(bot_config['symbols']),
        bot_config['starting_currency'] * len(bot_config['symbols'])
    )
    balance_queries.append(query)
    for symbol in bot_config['symbols']:
        query = balance_query.format(id, symbol, 0.0, 0.0, 0.0)
        balance_queries.append(query)

        # Create currency pair config
        query = currency_pair_config_query.format(
            id,
            f"{symbol}{bot_config['currency']}",
            bot_config['name'].format(id, symbol),
            bot_config['exchange'],
            interval,
            strategy_type,
            bot_config['starting_currency'],
            bot_config['starting_currency']
        )
        currency_pair_config_queries.append(query)
    
        # Create signal
        query = signal_query.format(config_id)
        signal_queries.append(query)

        # Create strategy config
        for key, value in params.items():
            query = strategy_config_query.format(config_id, strategy_type, key, value)
            strategy_config_queries.append(query)
        
        config_id += 1
        

with open('bot.testing.sql', 'w+') as f:
    for query in user_queries:
        f.write(query + '\n')
    
    for query in api_key_queries:
        f.write(query + '\n')
    
    for query in portfolio_queries:
        f.write(query + '\n')
    
    for query in balance_queries:
        f.write(query + '\n')
    
    for query in currency_pair_config_queries:
        f.write(query + '\n')
    
    for query in signal_queries:
        f.write(query + '\n')
    
    for query in strategy_config_queries:
        f.write(query + '\n')

#### Invalid symbols for USDT

In [20]:
invalid_symbols = ['BTCDOMUSDT', 'LUNA2USDT', 'LUNAUSDT', 'BLUEBIRDUSDT', 'DEFIUSDT', '1000SHIBUSDT', 'MANAUSDT', '1000LUNCUSDT', 'BLURUSDT', 'FOOTBALLUSDT', '1000PEPEUSDT', '1000FLOKIUSDT', '1000XECUSDT']

## SQL Debugging and fixing

In [12]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
from src.service import DataService

ENGINE  = create_engine(app_config.SQLALCHEMY_DATABASE_URI, echo=False, pool_size=20, max_overflow=0)

**Updating invalid strategy in the strategy configs**

In [29]:
with Session(ENGINE) as session:
    bots = DataService().get_bots(session, '45')
    for bot in bots:
        configs = bot.strategy_config
        for config in configs:
            config.strategy = StrategyType.CHAIKIN_OSCILLATOR.name
            # session.flush()

    # session.commit()

**Deleting invalid symbols**

In [21]:
with ENGINE.connect() as conn:
    for currency_pair in invalid_symbols:
        symbol = currency_pair[:-4]
        conn.execute(text(f"DELETE FROM symbols WHERE symbol = '{symbol}'"))
        conn.execute(text(f"DELETE FROM currency_pair_configs WHERE currency_pair = '{currency_pair}'"))
        conn.commit()